<center><h2>Real world application: linear regression</h2></center>

# #1. reset python kernel

In [1]:
%reset -f

---

# #2. import packages

In [2]:
import numpy             as np
import statsmodels.api   as sm
import pandas            as pd
import matplotlib.pyplot as plt
import statsmodels.api   as sm
import seaborn           as sns

from sklearn.linear_model       import LassoLarsCV, LassoCV, ElasticNetCV
from solar                      import solar
from statsmodels.iolib.summary2 import summary_col
from collections                import OrderedDict

---

# #3. load house price data

In [3]:
linTB = pd.read_csv("Data2010.csv")

## #3(a). rename variables

In [4]:
linTB.rename(columns={'Tot_P_P':'TotPop',
                      'Mean_age_persons':'AgeSA1',
                      'Australian_citizen_P':'Aus',
                      'Average_num_psns_per_bedroom':'psn-per-bedroom',
                      'Average_household_size':'household_size',
                      'FinalResultEventPrice': 'Price',
                      'Lang_spoken_home_Eng_only_P':'Eng-speak',
                      'Mean_mortgage_repay_monthly':'Mortgage',
                      'Mean_Tot_prsnl_inc_weekly':'Inc',
                      'Mean_rent_weekly':'Rent',
                      'Mean_Tot_fam_inc_weekly':'FamInc'}, 
                 inplace=True)
linTB = linTB.drop(['Unnamed: 0'],axis=1)
linTB = linTB.dropna()
linTB['const'] = 1
linTB.columns

#drop larger than 1500
#linTB['index'] = linTB['AreaSize'] <=4000
#linTB = linTB[linTB['index']==1]

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5', 'Price', 'const'],
      dtype='object')

In [5]:
linTB['AreaSize'].describe()

count    11974.000000
mean      1848.073242
std       4688.261157
min          0.000000
25%        478.000000
50%        724.000000
75%       1554.250000
max      63156.000000
Name: AreaSize, dtype: float64

In [6]:
linTB.to_csv('House2010_linear.csv')
Y_linear = linTB[['Price']]
X_linear = linTB.drop(columns=['Price'])
X_linear.columns

Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5', 'const'],
      dtype='object')

## check correlation among all variables

In [7]:
X_linear.drop(columns=['const']).corr()

,TotPop,Eng-speak,Aus,AgeSA1,Mortgage,Inc,Rent,FamInc,psn-per-bedroom,household_size,...,ReadingY3,WritingY3,SpellingY3,GrammarY3,NumeracyY3,ReadingY5,WritingY5,SpellingY5,GrammarY5,NumeracyY5
TotPop,1.000000,0.452736,0.814456,-0.204965,-0.059378,0.015972,0.041612,-0.053217,0.308566,-0.084312,...,-0.079804,-0.037282,0.014081,-0.058782,-0.058847,-0.119032,-0.087615,-0.040904,-0.102877,-0.047173
Eng-speak,0.452736,1.000000,0.689981,0.145351,0.254385,0.461019,0.179371,0.511514,-0.379772,-0.155546,...,0.205366,0.096137,-0.081108,0.221936,0.116435,0.293966,0.111116,-0.023205,0.255398,0.068249
Aus,0.814456,0.689981,1.000000,0.109320,0.024424,0.004614,0.000968,0.048556,-0.101535,0.121533,...,-0.048214,-0.050110,-0.019485,-0.011958,-0.059727,-0.042430,-0.070373,-0.057043,-0.044004,-0.055904
AgeSA1,-0.204965,0.145351,0.109320,1.000000,0.193832,-0.080814,0.046866,0.122437,-0.469739,0.196574,...,0.044367,0.040703,-0.021597,0.092318,0.022298,0.073198,0.019518,-0.005530,0.068482,0.005293
Mortgage,-0.059378,0.254385,0.024424,0.193832,1.000000,0.398752,0.387588,0.549591,-0.273102,0.081163,...,0.187181,0.122601,-0.008761,0.193886,0.123458,0.216119,0.088532,-0.006790,0.190665,0.089383
Inc,0.015972,0.461019,0.004614,-0.080814,0.398752,1.000000,0.444418,0.812792,-0.137790,-0.477492,...,0.351626,0.223391,-0.001620,0.318015,0.245047,0.392239,0.221610,0.042474,0.339091,0.176810
Rent,0.041612,0.179371,0.000968,0.046866,0.387588,0.444418,1.000000,0.489310,-0.089897,0.030407,...,0.189833,0.123867,0.034322,0.199260,0.129345,0.162227,0.087410,0.023125,0.171794,0.117234
FamInc,-0.053217,0.511514,0.048556,0.122437,0.549591,0.812792,0.489310,1.000000,-0.353859,-0.129558,...,0.382692,0.271322,0.022806,0.365709,0.291191,0.427464,0.242428,0.042307,0.377735,0.191875
psn-per-bedroom,0.308566,-0.379772,-0.101535,-0.469739,-0.273102,-0.137790,-0.089897,-0.353859,1.000000,-0.164361,...,-0.158331,-0.097002,0.066218,-0.196509,-0.096219,-0.246408,-0.122427,-0.007408,-0.225858,-0.058986
household_size,-0.084312,-0.155546,0.121533,0.196574,0.081163,-0.477492,0.030407,-0.129558,-0.164361,1.000000,...,-0.157798,-0.084310,0.014381,-0.128440,-0.101970,-0.166208,-0.083652,-0.039423,-0.159446,-0.066232


---

# #4. generate X and Y for variable selection

In [8]:
X_linear_input = np.array(X_linear)
Y_linear_input = np.array(Y_linear)

In [9]:
obs = X_linear_input.shape[0]

Y_linear_input.shape = (obs,)

---

# #5. compute CV coordinate descend (CV-cd), CV lars for lasso (CV-lars-lasso), CV elastic net (CV-en) and solar regression 

In [10]:
# CV coordinate descend (CV-cd)

RegLinear1 = LassoCV(cv=10, normalize = True, random_state=0)
RegLinear1.fit(X_linear_input, Y_linear_input)

LinearLassoCV_ceof = RegLinear1.coef_
LinearLassoCV_ind  = np.nonzero(LinearLassoCV_ceof)


# CV-lars-lasso

np.random.seed(0)

RegLinear2 = LassoLarsCV(cv=10)
RegLinear2.fit(X_linear_input,Y_linear_input)

LinearLasso_lars_ceof = RegLinear2.coef_
LinearLasso_lars_ind  = np.nonzero(LinearLasso_lars_ceof)

# CV-en

RegLinear3 = ElasticNetCV(l1_ratio=[.5, .7, .9, .95], normalize=True, random_state=0)
RegLinear3.fit(X_linear_input,Y_linear_input)

# solar

Y_linear_input.shape = (obs,1)

np.random.seed(0)

RegLinear4 = solar( X_linear_input,Y_linear_input, 10, -0.02, lasso = False)
LinearSolar_coef, LinearOpt_c, LinearTest_error, LinearQc_list, _, _, _, _ = RegLinear4.fit()

LinearSolar_ind = np.nonzero(LinearSolar_coef)[0]

---

# #8. the results of variable selection

## #6(c). variable selection by lasso, solar and elastic net

### variables selected by CV-lars-lasso

In [11]:
LinearLasso_lars_active = X_linear.columns[LinearLasso_lars_ind]
print(np.count_nonzero(LinearLasso_lars_ind))
print(LinearLasso_lars_active)

44
Index(['Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Rent', 'FamInc',
       'psn-per-bedroom', 'household_size', 'TVO2009', 'Suburb_Area',
       'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport', 'Beach',
       'Cemetery', 'ChildCare', 'Club', 'GolfCourt', 'High', 'Lib', 'Museum',
       'Park', 'Police', 'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat',
       'Rubbish', 'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary',
       'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3',
       'GrammarY3', 'NumeracyY3', 'WritingY5', 'SpellingY5', 'GrammarY5'],
      dtype='object')


### variables selected by CV-cd

In [12]:
LinearLassoCV_active = X_linear.columns[LinearLassoCV_ind]
print(np.count_nonzero(LinearLassoCV_ceof))
print(LinearLassoCV_active)

44
Index(['Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Rent', 'FamInc',
       'psn-per-bedroom', 'household_size', 'TVO2009', 'Suburb_Area',
       'AreaSize', 'Bedrooms', 'Baths', 'Parking', 'Airport', 'Beach',
       'Cemetery', 'ChildCare', 'Club', 'GolfCourt', 'High', 'Lib', 'Museum',
       'Park', 'Police', 'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat',
       'Rubbish', 'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary',
       'DistBound', 'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3',
       'GrammarY3', 'NumeracyY3', 'WritingY5', 'SpellingY5', 'GrammarY5'],
      dtype='object')


### variables selected by solar

In [13]:
LinearSolar_active = X_linear.columns[LinearSolar_ind]
print(np.count_nonzero(LinearSolar_coef))
print(LinearSolar_active)

9
Index(['Mortgage', 'Rent', 'FamInc', 'Bedrooms', 'Baths', 'Parking', 'Beach',
       'Gaol', 'ICSEA'],
      dtype='object')


### variables selected by CV-en

In [14]:
LinearEn_coef   = RegLinear3.coef_
LinearEn_ind    = np.nonzero(LinearEn_coef)
LinearEn_active = X_linear.columns[LinearEn_ind]
print(np.count_nonzero(LinearEn_coef))
print(LinearEn_active)

57
Index(['TotPop', 'Eng-speak', 'Aus', 'AgeSA1', 'Mortgage', 'Inc', 'Rent',
       'FamInc', 'psn-per-bedroom', 'household_size', 'TVO2010', 'TPO2010',
       'TVO2009', 'TPO2009', 'Suburb_Area', 'AreaSize', 'Bedrooms', 'Baths',
       'Parking', 'Airport', 'Beach', 'Cemetery', 'ChildCare',
       'CommunityFacility', 'Club', 'Gaol', 'GeneralHospital', 'GolfCourt',
       'High', 'Lib', 'MedCenter', 'Museum', 'Park', 'PO', 'Police',
       'PreSchool', 'PrimaryHigh', 'Primary', 'RailStat', 'Rubbish', 'Sewage',
       'SportsCenter', 'SportsCourtField', 'Swimming', 'Tertiary', 'DistBound',
       'ICSEA', 'ReadingY3', 'WritingY3', 'SpellingY3', 'GrammarY3',
       'NumeracyY3', 'ReadingY5', 'WritingY5', 'SpellingY5', 'GrammarY5',
       'NumeracyY5'],
      dtype='object')


# #9. Post-selection regression

## #7(a). post-elastic-net OLS

In [15]:
X_LinearEn   = X_linear[LinearEn_active]
X_LinearEn   = sm.add_constant(X_LinearEn)
OLS_LinearEn = sm.OLS(Y_linear, X_LinearEn)

OLS_LinearEn_result = OLS_LinearEn.fit()
print(OLS_LinearEn_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     254.5
Date:                Mon, 19 Apr 2021   Prob (F-statistic):               0.00
Time:                        04:16:39   Log-Likelihood:            -1.7341e+05
No. Observations:               11974   AIC:                         3.469e+05
Df Residuals:                   11916   BIC:                         3.474e+05
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -8.862e+05   1.87e+0

## #7(b). post-lasso (CV-cd) OLS

In [16]:
X_LinearLassoCV   = X_linear[LinearLassoCV_active]
Y_LinearLassoCV   = Y_linear
X_LinearLassoCV   = sm.add_constant(X_LinearLassoCV)
OLS_LinearLassoCV = sm.OLS(Y_LinearLassoCV, X_LinearLassoCV)

OLS_LinearLassoCV_result = OLS_LinearLassoCV.fit()
print(OLS_LinearLassoCV_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     328.5
Date:                Mon, 19 Apr 2021   Prob (F-statistic):               0.00
Time:                        04:16:39   Log-Likelihood:            -1.7342e+05
No. Observations:               11974   AIC:                         3.469e+05
Df Residuals:                   11929   BIC:                         3.473e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -8.274e+05   1.74e+05  

## #7(c). post-lasso (lars-lasso) OLS

In [17]:
X_LinearLars   = X_linear[LinearLasso_lars_active]
Y_LinearLars   = Y_linear
X_LinearLars   = sm.add_constant(X_LinearLars)
OLS_LinearLars = sm.OLS(Y_LinearLars, X_LinearLars)

OLS_LinearLars_result = OLS_LinearLars.fit()
print(OLS_LinearLars_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     328.5
Date:                Mon, 19 Apr 2021   Prob (F-statistic):               0.00
Time:                        04:16:39   Log-Likelihood:            -1.7342e+05
No. Observations:               11974   AIC:                         3.469e+05
Df Residuals:                   11929   BIC:                         3.473e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -8.274e+05   1.74e+05  

## #7(d). post-solar OLS

In [18]:
X_Gaol     = X_linear[LinearSolar_active]
X_Gaol     = sm.add_constant(X_Gaol)

OLS_LinearSolar  = sm.OLS(Y_linear, X_Gaol)

OLS_LinearSolar_result = OLS_LinearSolar.fit()

print(OLS_LinearSolar_result.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.494
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     1296.
Date:                Mon, 19 Apr 2021   Prob (F-statistic):               0.00
Time:                        04:16:39   Log-Likelihood:            -1.7410e+05
No. Observations:               11974   AIC:                         3.482e+05
Df Residuals:                   11964   BIC:                         3.483e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.486e+06   9.84e+04    -25.263      0.0

## hold-out average result

In [19]:
half_size = int(np.round(X_Gaol.shape[0]/2))

X_half_1 = X_Gaol[0:half_size]
Y_half_1 = Y_linear[0:half_size]
X_half_2 = X_Gaol[half_size:]
Y_half_2 = Y_linear[half_size:]

active_var = [str(i) for i in LinearSolar_active]
XX_half_1 = X_half_1[LinearSolar_active]
YY_half_1 = Y_half_1
XX_half_1 = sm.add_constant(XX_half_1)
reg_half_1 = sm.OLS(YY_half_1, XX_half_1)

result_half_1 = reg_half_1.fit()

XX_half_2 = X_half_2[LinearSolar_active]
YY_half_2 = Y_half_2
XX_half_2 = sm.add_constant(XX_half_2)
reg_half_2 = sm.OLS(YY_half_2, XX_half_2)

result_half_2 = reg_half_2.fit()

In [20]:
#hold-out average R^2
average_R2 = (result_half_1.rsquared + result_half_2.rsquared)/2

#hold-out average t-value
Mortgage_t = (result_half_1.tvalues[1]  + result_half_2.tvalues[1] )/2
Rent_t     = (result_half_1.tvalues[2]  + result_half_2.tvalues[2] )/2
FamInc_t   = (result_half_1.tvalues[3]  + result_half_2.tvalues[3] )/2
Bedrooms_t = (result_half_1.tvalues[4]  + result_half_2.tvalues[4] )/2
Baths_t    = (result_half_1.tvalues[5]  + result_half_2.tvalues[5] )/2
Parking_t  = (result_half_1.tvalues[6]  + result_half_2.tvalues[6] )/2
Beach_t    = (result_half_1.tvalues[7]  + result_half_2.tvalues[7] )/2
Gaol_t     = (result_half_1.tvalues[8]  + result_half_2.tvalues[8] )/2
ICSEA_t    = (result_half_1.tvalues[9]  + result_half_2.tvalues[9] )/2

#hold-out average p-value
Mortgage_p = (result_half_1.pvalues[1]  + result_half_2.pvalues[1] )/2
Rent_p     = (result_half_1.pvalues[2]  + result_half_2.pvalues[2] )/2
FamInc_p   = (result_half_1.pvalues[3]  + result_half_2.pvalues[3] )/2
Bedrooms_p = (result_half_1.pvalues[4]  + result_half_2.pvalues[4] )/2
Baths_p    = (result_half_1.pvalues[5]  + result_half_2.pvalues[5] )/2
Parking_p  = (result_half_1.pvalues[6]  + result_half_2.pvalues[6] )/2
Beach_p    = (result_half_1.pvalues[7]  + result_half_2.pvalues[7] )/2
Gaol_p     = (result_half_1.pvalues[8]  + result_half_2.pvalues[8] )/2
ICSEA_p    = (result_half_1.pvalues[9]  + result_half_2.pvalues[9] )/2

#hold-out average se
Mortgage_se = (result_half_1.bse[1]  + result_half_2.bse[1] )/2
Rent_se     = (result_half_1.bse[2]  + result_half_2.bse[2] )/2
FamInc_se   = (result_half_1.bse[3]  + result_half_2.bse[3] )/2
Bedrooms_se = (result_half_1.bse[4]  + result_half_2.bse[4] )/2
Baths_se    = (result_half_1.bse[5]  + result_half_2.bse[5] )/2
Parking_se  = (result_half_1.bse[6]  + result_half_2.bse[6] )/2
Beach_se    = (result_half_1.bse[7]  + result_half_2.bse[7] )/2
Gaol_se     = (result_half_1.bse[8]  + result_half_2.bse[8] )/2
ICSEA_se    = (result_half_1.bse[9]  + result_half_2.bse[9] )/2

df_hor = pd.concat([pd.DataFrame({'variable name':active_var}), 
                    pd.DataFrame({'se':[Mortgage_se, Rent_se, FamInc_se, Bedrooms_se, Baths_se,
                                        Parking_se, Beach_se, Gaol_se, ICSEA_se],
                                  't-val':[Mortgage_t, Rent_t, FamInc_t, Bedrooms_t, Baths_t,
                                           Parking_t, Beach_t, Gaol_t, ICSEA_t], 
                                  'p-val': [Mortgage_p, Rent_p, FamInc_p, Bedrooms_p, Baths_p, 
                                            Parking_p, Beach_p, Gaol_p, ICSEA_p]})], 
                   axis=1, join='inner')

df_hor.round(3)

,variable name,se,t-val,p-val
0,Mortgage,11.277,15.814,0.000
1,Rent,51.332,7.337,0.000
2,FamInc,16.833,5.192,0.029
3,Bedrooms,8476.450,20.301,0.000
4,Baths,11646.107,17.831,0.000
5,Parking,9840.929,9.947,0.000
6,Beach,228847.145,-3.052,0.139
7,Gaol,166526.952,-12.709,0.000
8,ICSEA,139.440,11.730,0.000


In [21]:
print("the hold-out average R2 is ", average_R2.round(2))

the hold-out average R2 is  0.5


## #7(e). generate a comparison table for post-selection OLS result (slope, significance, $R^2$)

In [22]:
info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}

results_table = summary_col(results=[OLS_LinearEn_result, OLS_LinearLassoCV_result, 
                                     OLS_LinearLars_result, OLS_LinearSolar_result],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['post-en',
                                         'post-lasso(lars)',
                                         'post-larsso(cd)',
                                         'post-solar'],
                            info_dict=info_dict,
                            regressor_order=['const',
                                             'Bedrooms',
                                             'Baths',
                                             'Parking',
                                             'Beach',
                                             'Gaol',
                                             'ICSEA',
                                             'Mortage',
                                             'Rent',
                                             'FamInc'])

results_table.add_title('Table 1 - Post-selection OLS comparison')

print(results_table)

                    Table 1 - Post-selection OLS comparison
                     post-en     post-lasso(lars) post-larsso(cd)   post-solar  
--------------------------------------------------------------------------------
const             -886234.41***  -827387.50***    -827387.50***   -2486422.15***
                  (186680.91)    (174136.59)      (174136.59)     (98422.98)    
Bedrooms          165639.00***   166225.82***     166225.82***    169510.52***  
                  (6433.57)      (6404.82)        (6404.82)       (6015.37)     
Baths             210101.83***   210600.58***     210600.58***    209626.52***  
                  (8115.07)      (8048.92)        (8048.92)       (8297.24)     
Parking           97790.57***    96883.13***      96883.13***     97623.23***   
                  (6689.35)      (6688.13)        (6688.13)       (6985.67)     
Beach             -5029681.75*** -4051671.59***   -4051671.59***  -796281.77*** 
                  (600061.27)    (262369.00)     

In [23]:
!rm -rf Application_Houseprice_2010_linear.html
!jupyter nbconvert --to html Application_Houseprice_2010_linear.ipynb 

[NbConvertApp] Converting notebook Application_Houseprice_2010_linear.ipynb to html
[NbConvertApp] Writing 705279 bytes to Application_Houseprice_2010_linear.html
